# Marknadsanalys av diamanter



# Inledning

## Guldfynds inköpsavdelning har under en tid haft svårt att hitta ädelstenar som blir rätt prissatta ut till handeln. Detta har lett till sviktande marginaler. Denna analys är ett jobb beställt av Guldfynd och ska presenteras utav deras Sourcing Manager till ledningsgruppen.

## I denna analys kommer vi att titta på diamanter – världens kanske mest eftertraktade ädelsten.

Diamantpriser påverkas av flera faktorer: vikt (carat), slipning (cut), färg (color) och klarhet (clarity). Dessa egenskaper påverkar inte bara stenens utseende utan också dess marknadsvärde. Det är därför avgörande för Guldfynd att förstå vilka kombinationer av dessa egenskaper som erbjuder bäst **värde för pengarna**, snarare än att enbart köpa in det som är "bäst" på pappret.

Den här analysen syftar till att:
- Identifiera **vilka diamantsegment som ger hög kvalitet till ett rimligt pris**.
- Ge **konkreta rekommendationer** till inköpschefen om vilka stenar som bör prioriteras.
- Tydliggöra **prisdrivande faktorer**, samt visa var företaget kan hitta **potentiella marginalfördelar**.

Målet är alltså att ge Guldfynd ett **beslutsunderlag** som bidrar till bättre affärsmässighet i inköpen, högre marginaler och en konkurrenskraftig produktportfölj.

Analysen utgår ifrån ett större dataset med över 54 000 diamanter, där varje rad representerar en unik sten med olika egenskaper och ett noterat marknadsvärde.



## Inläsning och rensning av data
- I detta avsnittt läser vi in datasetet diamonds.csv i våran notebook. Vi listar hur många rader och vad det är för datatyper i setet.
- Vi importerar också våra tredjepartsbibliotek Pandas, Numpy och Plotly.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

#Läser in dataset
df = pd.read_csv("diamonds.csv")

#längd på dataframe
length_df = len(df)
print(f'Dataset length: {length_df}')

#information om datatyper och columner
df.info()



Dataset length: 53940
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53918 non-null  float64
 2   cut         53939 non-null  object 
 3   color       53936 non-null  object 
 4   clarity     53910 non-null  object 
 5   depth       53934 non-null  float64
 6   table       53935 non-null  float64
 7   price       53928 non-null  float64
 8   x           53937 non-null  float64
 9   y           53936 non-null  float64
 10  z           53937 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 4.5+ MB


#### I detta avsnitt nedan så gör vi en funktion för att sortera och göra en data-cleanse på datasetet.


In [38]:
from pandas.api.types import CategoricalDtype

def clean_diamond_data(df):
    total_start = df.shape[0]
    removed = {}

    if 'index' not in df.columns:
        df.reset_index(inplace=True)

    
    before = df.shape[0]
    df = df.dropna(subset=['index', 'cut', 'color', 'clarity', 'price', 'carat', 'x', 'y', 'z', 'depth'])
    removed["Saknade värden"] = before - df.shape[0]

    
    numeric_cols = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
    before = df.shape[0]
    for col in numeric_cols:
        df = df[df[col] > 0]
    removed["Nollvärden i numeriska kolumner"] = before - df.shape[0]

   
    before = df.shape[0]
    df = df[(df['x'] <= 15) & (df['y'] <= 15) & (df['z'] <= 15)]
    removed["Extrema mått (>15 mm)"] = before - df.shape[0]

    
    before = df.shape[0]
    df = df[~((df['carat'] < 1) & (df['z'] > 10))]
    removed["Misstänkt djup (carat < 1 & z > 10)"] = before - df.shape[0]

    
    df['depth_calc'] = (df['z'] / ((df['x'] + df['y']) / 2)) * 100
    df['depth_diff'] = abs(df['depth_calc'] - df['depth'])
    before = df.shape[0]
    df = df[df['depth_diff'] <= 0.5]
    removed[">1% avvikelse i depth"] = before - df.shape[0]

   
    df.drop(columns=["depth_calc", "depth_diff"], inplace=True)

    df['index'] = df['index'].astype(int)

   
    df = df.reset_index(drop=True)

    df.dropna()

    print("🧹 Sammanställning av borttagna rader:")
    for reason, quant in removed.items():
        print(f"- {reason}: {quant} rader")

    total_borttagna = total_start - df.shape[0]
    print(f"\nTotalt borttagna rader: {total_borttagna}")
    print(f"✅ Rader kvar efter rensning: {df.shape[0]}")

    return df

df = clean_diamond_data(df)

🧹 Sammanställning av borttagna rader:
- Saknade värden: 74 rader
- Nollvärden i numeriska kolumner: 22 rader
- Extrema mått (>15 mm): 3 rader
- Misstänkt djup (carat < 1 & z > 10): 0 rader
- >1% avvikelse i depth: 150 rader

Totalt borttagna rader: 249
✅ Rader kvar efter rensning: 53691


## 📊 Datastädning – Förutsättning för en tillförlitlig analys

Innan vi påbörjar någon analys av diamantdata behöver vi säkerställa att datan vi arbetar med är **komplett, rimlig och representativ** för verkliga köpbeslut. Rådata innehåller ofta saknade värden, felaktiga inmatningar och orimliga mått – allt detta kan snedvrida analysresultaten och därmed leda till felaktiga rekommendationer.

### 🔍 Vad gör `clean_diamond_data()`?

Funktionen `clean_diamond_data(df)` genomför en noggrann rensning av datan i flera steg:

1. **Tar bort rader med saknade värden** i kritiska kolumner som vikt (carat), mått (x, y, z), slipning (cut), färg, klarhet och pris.
2. **Filtrerar bort rader där numeriska värden är noll eller negativa**, vilket inte är fysiskt möjligt för ädelstenar.
3. **Tar bort extrema mått** där någon av längd, bredd eller djup överstiger 15 mm. Detta identifierar outliers som inte speglar kommersiellt gångbara diamanter.
4. **Tar bort orimliga kombinationer** såsom små stenar (carat < 1) med mycket stor höjd (z > 10 mm), vilket ofta kan indikera felregistrering av datan.
5. **Jämför uppmätt djup med beräknat geometriskt djup** och tillåter endast en avvikelse på max ±1%. Detta är ett kvalitetsfilter som upptäcker inkonsekventa inmatningar.
6. **Konverterar kvalitativa egenskaper** som `cut`, `color` och `clarity` till ordnade kategorier. Detta gör visualiseringar och gruppanalyser mer begripliga och meningsfulla.

### 📈 Vad visar resultatet?

Efter rensningen skrivs en sammanställning ut i konsolen som visar:

- Hur många rader som tagits bort i varje steg
- Totalt antal borttagna rader
- Hur många rader som återstår i den slutgiltiga, städade datamängden

Detta gör rensningen **transparent och spårbar**, vilket är viktigt för att ledningen ska kunna förstå och lita på analysens grunddata.

Den städade datan kan nu användas för att identifiera lönsamma diamantsegment utan att störas av felaktigheter eller extremvärden.

---


### 🔄 Anpassning till GIA – Cut Mapping

I vårt dataset ser vi slipkvaliteter som "Ideal", "Premium", "Very Good" etc. Dessa används ofta av återförsäljare eller icke-certifierade aktörer och tenderar att vara mer generösa i sin bedömning än GIA:s formella skala.

För att göra vår analys mer realistisk och jämförbar med GIA-certifikat har vi därför översatt dessa till GIA-standard enligt följande:

| Datasetets värde | GIA-standard     |
|------------------|------------------|
| Ideal            | Excellent         |
| Premium          | Very Good         |
| Very Good        | Good              |
| Good             | Fair              |
| Fair             | Poor              |

Syftet är att inte överskatta kvaliteten på diamanterna i vår analys. Detta gör resultaten mer användbara för faktiska inköpsbeslut, där certifikat ofta krävs.



I nästa avsnitt ska vi mappa om alla datasetets värden till GIA standard med python-kod

In [7]:
cut_mapping = {
    "Ideal": "Excellent",
    "Premium": "Very Good",
    "Very Good": "Good",
    "Good": "Fair",
    "Fair": "Poor"
}

df['cut_mapped'] = df['cut'].map(cut_mapping)

cut_order_gia = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']
df['cut_mapped'] = df['cut_mapped'].astype(CategoricalDtype(categories=cut_order_gia, ordered=True))

## Överblick
#### I detta avsnittet ska vi skapa en visuell överblick över hur priset är fördelat på de olika stenarna

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, subplot_titles=("Pris vs Carat", "Histogram över Carat"))


fig.add_trace(
    go.Scatter(
        x=df["carat"],
        y=df["price"],
        mode='markers',
        marker=dict(color='royalblue', opacity=0.6, size=3),
        name='Pris vs Carat'
    ),
    row=1, col=1
)


fig.add_trace(
    go.Histogram(
        x=df["carat"],
        nbinsx=30,
        marker_color='mediumturquoise',
        name='Histogram Carat'
    ),
    row=1, col=2
)


fig.update_layout(
    title_text="Diamantdata: Pris och Carat",
    showlegend=False,
    height=500,
    width=1000,
    plot_bgcolor='white'
)


fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')

fig.show()


### Vad vi ser

I de två översiktsdiagrammen framgår det tydligt att majoriteten av diamanterna i datasetet väger under 1 carat. Detta bekräftas i histogrammet till höger, där de flesta observationerna är koncentrerade i de lägre caratintervallen. Distributionen är tydligt snedfördelad åt höger, vilket innebär att större diamanter är ovanligare.

Scatterplotten visar relationen mellan pris och carat. Det finns ett tydligt positivt samband mellan dessa två variabler – priset ökar generellt med ökad vikt. 

Vi kan även se att vid vissa specifika viktgränser, såsom **1.0, 1.5 och 2.0 carat**, tenderar priserna att vara både högre och mer utspridda jämfört med närliggande caratnivåer. 

### Varför detta görs

Syftet med dessa visualiseringar är att snabbt få en **överblick över hur diamanterna i datamängden är fördelade**, samt att identifiera **grundläggande samband och avvikelser**. Detta ger värdefull insikt både för:

- **Analytiker**, som vill skapa modeller för prissättning eller efterfrågeprognoser
- **Beslutsfattare**, som vill förstå vilka typer av diamanter som är vanligast och vilka som driver upp priset

### Nästa steg

I nästa steg ska jag presentera data över stenar med karat som används i smycken med störst volym på den Skandinaviska marknaden.

In [41]:
df_filtered = df[(df['carat'] >= 0.25) & (df['carat'] <=1.0)]

fig3 = px.scatter(df_filtered,
                  x ='carat',
                  y ='price', 
                  title = 'Pris vs Karat (0.25 till 1.00 ct)',
                  color='cut',
                  labels={'carat': "Carat", 'price': "Price"})

fig4 = px.histogram(df_filtered,
                    x='carat',
                    nbins=30,
                    title='Antal diamanter i intervallet 0.25-1.00 ct',
                    labels={'carat': "Carat", 'count': "Antal"},
                    color_discrete_sequence=['mediumturquoise'])

fig_combined = make_subplots(rows=1, cols = 2,
                             subplot_titles=("Pris vs Carat", "Antal Diamanter per Carat"),
                             column_widths=[0.5,0.5])

fig_combined.add_trace(fig3['data'][0],row=1,col=1)
for trace in fig4['data']:
    fig_combined.add_trace(trace, row=1, col=2)

fig_combined.update_layout(height=500, width=1000, showlegend=False)
fig_combined.show()



### Vad vi ser
I scatter-diagrammet till vänster ser vi att diamantens pris ökar tydligt med carat-vikten inom intervallet 0.25–1.00 ct. Speciellt nära 1.00 ct syns en koncentration av både fler observationer och högre prisvariation, vilket indikerar ett psykologiskt eller marknadsmässigt tröskelvärde.

I histogrammet till höger framgår att de vanligaste carat-vikterna i detta intervall är 0.3 ct, 0.4 ct och runt 0.7 ct. Dessa toppar representerar troligen populära storlekar i smyckesproduktion, särskilt inom förlovnings- och vigselringar i Skandinavien.

### Varför görs detta
Syftet är att analysera de vanligaste och mest efterfrågade diamantvikterna inom ett relevant urval (0.25–1.00 ct), vilket är vanligt i smyckesindustrin. Genom att koppla detta till prisdata kan vi dra slutsatser om marknadsbeteenden, prissättning och potentiella kundpreferenser.

### Rekommendationer
* Fokusera på inköp och lager av diamanter i storlekarna 0.3–0.75 ct, då dessa verkar ha både hög efterfrågan och hanterbara priser.

* Undvik att göra inköp av stenar med cirka 1.00 ct utan prisanalys av segmentet, eftersom dessa ser ut att innebära en parabolisk prisökning, vilket i sin tur kan leda till marginalsänkning.

### Nästa steg 
Analysera vidare om prisvariationer vid 0.7 och upp till 1.5 ct beror på slipkvalitet, färg eller klarhet för att optimera värde för kund och marginal.



In [51]:
# Skapa två grupper: små och större diamanter
df_filtered = df[((df['carat'] >= 0.1) & (df['carat'] <= 0.75)) | 
                 ((df['carat'] >= 0.76) & (df['carat'] <= 1.5))]

# Lägg till storlekskategori
df_filtered["size_group"] = df_filtered["carat"].apply(lambda x: "0.2–0.75 ct" if x <= 0.75 else "0.76–1.5 ct")

# Boxplot: Prisvariation efter Slipkvalitet (Cut)
fig1 = px.scatter(df_filtered,
              x="carat",
              y="price",
              color="size_group",
              title="Prisvariation efter carat, minskad skala till 0.2 till 1.5ct",
              labels={"cut": "Slipkvalitet", "price": "Pris (USD)", "size_group": "Carat-grupp"})

fig1.show()

/var/folders/w3/_fr5m5_x717cxnsv2bczt3d40000gq/T/ipykernel_2993/1827331877.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Vad vi ser

I scatterploten över diamantdatasetet kan vi se att priset är relativt jämnt för diamanter med karatvärde mellan 0.2 och cirka 0.9. När vi når 1 karat och över, ökar prisets spridning markant. Till exempel har diamanter på 1 karat priser som sträcker sig från cirka 1600 till 16000, vilket visar en betydligt större volatilitet och variation i pris. Här får vi ha i åtanke att vi har inte tagit slipning, klarhet eller färg i beaktning.

### Varför detta görs

Syftet med denna visualisering är att förstå hur priset på diamanter varierar i relation till deras storlek (karat). Genom att identifiera var priset är relativt stabilt och var det blir mer volatilt kan vi bättre analysera marknadens dynamik och identifiera potentiella prismönster eller segment.

### Rekommendationer

* Fokusera på att noggrant analysera diamanter över 1 karat eftersom prisvariationen är stor, vilket kan påverka både prissättning och lagerhantering.
* Överväg att segmentera marknaden efter karatstorlek för att bättre rikta marknadsföring och försäljning.
* Vid inköp, var extra noggrann med kvalitetsparametrar som klarhet och slipning, då de sannolikt påverkar priset mer kraftigt vid högre karat.
